In [7]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [9]:
with open('credential.json') as f:
    data = json.load(f)
    client_id = data['client_id']
    client_secret = data['client_secret']

In [10]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

In [13]:
# Spotify object to access API
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [28]:
result = sp.search("John Mayer")

In [42]:
result['tracks']['items'][1]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/0hEurMDQu99nJRq8pTxO14'},
  'href': 'https://api.spotify.com/v1/artists/0hEurMDQu99nJRq8pTxO14',
  'id': '0hEurMDQu99nJRq8pTxO14',
  'name': 'John Mayer',
  'type': 'artist',
  'uri': 'spotify:artist:0hEurMDQu99nJRq8pTxO14'}]

### Extract Spotify Albums

In [45]:
#Extract Artist's uri
artist_uri = result['tracks']['items'][1]['artists'][0]['uri']
#Pull all of the artist's albums
sp_albums = sp.artist_albums(artist_uri, album_type='album')
#Store artist's albums' names' and uris in separate lists
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])

In [46]:
album_names

['The Search for Everything',
 'Paradise Valley',
 'Born and Raised',
 'Battle Studies',
 'Where the Light Is: John Mayer Live In Los Angeles',
 'Continuum',
 'As/Is',
 'As/Is: Cleveland/Cincinnati, OH - 8/03-8/04/04',
 'As/Is: Houston, TX - 7/24/04',
 'As/Is: Philadelphia, PA/Hartford, CT - 8/14-8/15/04',
 'As/Is: Houston, TX - 7/24/04',
 'As/Is: Mountain View, CA - 7/16/04',
 'Heavier Things',
 'Any Given Thursday',
 'Any Given Thursday',
 'Room For Squares',
 'Inside Wants Out']

### Get Tracks

In [47]:
def albumSongs(uri):
    album = uri #assign album uri to a_name
    spotify_albums[album] = {} #Creates dictionary for that specific album
    #Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album'] = [] #create empty list
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    tracks = sp.album_tracks(album) #pull data on album tracks
    for n in range(len(tracks['items'])): #for each song track
        spotify_albums[album]['album'].append(album_names[album_count]) #append album name tracked via album_count
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

In [48]:
spotify_albums = {}
album_count = 0
for i in album_uris: #each album
    albumSongs(i)
    print("Album " + str(album_names[album_count]) + " songs has been added to spotify_albums dictionary")
    album_count+=1 #Updates album count once all tracks have been added

Album The Search for Everything songs has been added to spotify_albums dictionary
Album Paradise Valley songs has been added to spotify_albums dictionary
Album Born and Raised songs has been added to spotify_albums dictionary
Album Battle Studies songs has been added to spotify_albums dictionary
Album Where the Light Is: John Mayer Live In Los Angeles songs has been added to spotify_albums dictionary
Album Continuum songs has been added to spotify_albums dictionary
Album As/Is songs has been added to spotify_albums dictionary
Album As/Is: Cleveland/Cincinnati, OH - 8/03-8/04/04 songs has been added to spotify_albums dictionary
Album As/Is: Houston, TX - 7/24/04 songs has been added to spotify_albums dictionary
Album As/Is: Philadelphia, PA/Hartford, CT - 8/14-8/15/04 songs has been added to spotify_albums dictionary
Album As/Is: Houston, TX - 7/24/04 songs has been added to spotify_albums dictionary
Album As/Is: Mountain View, CA - 7/16/04 songs has been added to spotify_albums diction

### Get audio features from each song

In [50]:
def audio_features(album):
    #Add new key-values to store audio features
    spotify_albums[album]['acousticness'] = []
    spotify_albums[album]['danceability'] = []
    spotify_albums[album]['energy'] = []
    spotify_albums[album]['instrumentalness'] = []
    spotify_albums[album]['liveness'] = []
    spotify_albums[album]['loudness'] = []
    spotify_albums[album]['speechiness'] = []
    spotify_albums[album]['tempo'] = []
    spotify_albums[album]['valence'] = []
    spotify_albums[album]['popularity'] = []
    #create a track counter
    track_count = 0
    for track in spotify_albums[album]['uri']:
        #pull audio features per track
        features = sp.audio_features(track)
        
        #Append to relevant key-value
        spotify_albums[album]['acousticness'].append(features[0]['acousticness'])
        spotify_albums[album]['danceability'].append(features[0]['danceability'])
        spotify_albums[album]['energy'].append(features[0]['energy'])
        spotify_albums[album]['instrumentalness'].append(features[0]['instrumentalness'])
        spotify_albums[album]['liveness'].append(features[0]['liveness'])
        spotify_albums[album]['loudness'].append(features[0]['loudness'])
        spotify_albums[album]['speechiness'].append(features[0]['speechiness'])
        spotify_albums[album]['tempo'].append(features[0]['tempo'])
        spotify_albums[album]['valence'].append(features[0]['valence'])
        #popularity is stored elsewhere
        pop = sp.track(track)
        spotify_albums[album]['popularity'].append(pop['popularity'])
        track_count+=1

In [51]:
import time
import numpy as np
sleep_min = 2
sleep_max = 5
start_time = time.time()
request_count = 0
for i in spotify_albums:
    audio_features(i)
    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " playlists completed")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))

retrying ...1secs
5 playlists completed
Loop #: 5
Elapsed Time: 15.63957405090332 seconds
10 playlists completed
Loop #: 10
Elapsed Time: 29.60930109024048 seconds
15 playlists completed
Loop #: 15
Elapsed Time: 42.77843379974365 seconds


### Create Dataframe Format Data

In [52]:
dic_df = {}
dic_df['album'] = []
dic_df['track_number'] = []
dic_df['id'] = []
dic_df['name'] = []
dic_df['uri'] = []
dic_df['acousticness'] = []
dic_df['danceability'] = []
dic_df['energy'] = []
dic_df['instrumentalness'] = []
dic_df['liveness'] = []
dic_df['loudness'] = []
dic_df['speechiness'] = []
dic_df['tempo'] = []
dic_df['valence'] = []
dic_df['popularity'] = []
for album in spotify_albums: 
    for feature in spotify_albums[album]:
        dic_df[feature].extend(spotify_albums[album][feature])
        
len(dic_df['album'])

206

In [53]:
import pandas as pd
df = pd.DataFrame.from_dict(dic_df)
df

,album,track_number,id,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,The Search for Everything,1,1LM6t24SjQr2bJHqeGIR4U,Still Feel Like Your Man,spotify:track:1LM6t24SjQr2bJHqeGIR4U,0.54700,0.858,0.6000,0.001130,0.1320,-6.677,0.0630,106.023,0.513,68
1,The Search for Everything,2,5ddXMXmXZ2FN4iliTG20nO,Emoji of a Wave,spotify:track:5ddXMXmXZ2FN4iliTG20nO,0.77100,0.494,0.3650,0.000978,0.1390,-8.920,0.0279,165.766,0.191,66
2,The Search for Everything,3,701DK0It9f7iurRnzKvF0y,Helpless,spotify:track:701DK0It9f7iurRnzKvF0y,0.15100,0.761,0.5980,0.002590,0.0314,-5.838,0.0268,103.978,0.937,65
3,The Search for Everything,4,0oiCeOPwm4zhwfyIpAE37y,Love on the Weekend,spotify:track:0oiCeOPwm4zhwfyIpAE37y,0.62300,0.718,0.5830,0.000662,0.0612,-7.884,0.0288,119.970,0.422,70
4,The Search for Everything,5,77Y57qRJBvkGCUw9qs0qMg,In the Blood,spotify:track:77Y57qRJBvkGCUw9qs0qMg,0.38600,0.546,0.6570,0.000010,0.0983,-6.526,0.0255,80.007,0.520,69
5,The Search for Everything,6,6eIygPhGcBFKKcxlGTJlY0,Changing,spotify:track:6eIygPhGcBFKKcxlGTJlY0,0.58700,0.444,0.3360,0.002740,0.1320,-7.778,0.0282,183.345,0.392,63
6,The Search for Everything,7,1G4s7WkhgmdQvYaGCHjw1G,"Theme from ""The Search for Everything""",spotify:track:1G4s7WkhgmdQvYaGCHjw1G,0.89700,0.595,0.3370,0.935000,0.1210,-11.060,0.0263,90.973,0.362,59
7,The Search for Everything,8,1crSsvtU9wZB15dIQXV2QH,Moving On and Getting Over,spotify:track:1crSsvtU9wZB15dIQXV2QH,0.59100,0.850,0.4610,0.040900,0.1000,-9.018,0.0539,94.038,0.628,65
8,The Search for Everything,9,3TQbr3G3U5wlwEJejmqC1F,Never on the Day You Leave,spotify:track:3TQbr3G3U5wlwEJejmqC1F,0.73000,0.535,0.3540,0.000012,0.1380,-8.610,0.0260,139.889,0.220,63
9,The Search for Everything,10,5KsLlcmWDoHUoJFzRw14wD,Rosie,spotify:track:5KsLlcmWDoHUoJFzRw14wD,0.48900,0.758,0.4650,0.000052,0.1160,-6.547,0.0374,138.902,0.759,65


In [55]:
# Remove duplicate data
print(len(df))
final_df = df.sort_values('popularity', ascending=False).drop_duplicates('name').sort_index()
print(len(final_df))

206
166
